### Libaries

In [1]:
import robotic as ry
import numpy as np
import time
from scipy.interpolate import Rbf
from scipy.interpolate import CubicSpline
print('version:', ry.__version__, ry.compiled())

version: 0.1.0 compile time: Oct 31 2023 17:13:07


### Initalise configuration

In [2]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))
C.view()

0

### Define color map

In [4]:
color_map = [[1,0,.5], # pink
             [0.5,1,0], # lime green
             [.5,1,1], # light turqois
             [0.5,0,1], # violet
             [1,0,0], # red
             [0,1,0], # green
             [0,0,1], # blue
             [1,.5,0], # orange
             [1,1,.5], # lime yellow
             [1,.5,1], # light pink                            
]

### Gripper functions

In [27]:
def gripper_open(C, bot, widht = 0.75, speed = 0.2):
    bot.gripperMove(ry._left, widht, speed)

    while not bot.gripperDone(ry._left):
        bot.sync(C, .1)

def gripper_close(bot, widht = 0.01, speed = 0.2):
    bot.gripperMove(ry._left, widht, speed)

    while not bot.gripperDone(ry._left):
        bot.sync(C, .1)

def gripper_close_grasp(bot, target_obj = '', force=10.00, widht = 0.01, speed = 0.2):
    bot.gripperCloseGrasp(ry._left, target_obj, force, widht, speed)
    while not bot.gripperDone(ry._left):
        bot.sync(C, .1)
    

### Create N number of building cuboids

In [6]:
def create_n_boxes(C, N, position='fixed'):

    for i in range(N):
        box_name = 'box{}'.format(i + 1)

        if position == 'fixed':
            position_val1 = 0.1 * (i - 1)
            C.addFrame(box_name) \
                .setPosition([position_val1, 0.35, 0.70]) \
                .setShape(ry.ST.ssBox, size=[0.05, 0.05, 0.10, 0.005]) \
                .setColor(color_map[i % len(color_map)]) \
                .setContact(True) \
                .setMass(1e-2)


        elif position == 'random':
            C.addFrame(box_name) \
                .setPosition([np.random.uniform(-0.5, 0.5), np.random.uniform(-0.5, 0.5), 0.70]) \
                .setShape(ry.ST.ssBox, size=[0.05, 0.05, 0.10, 0.005]) \
                .setColor(color_map[i % len(color_map)]) \
                .setContact(True) \
                .setMass(1e-2)
    
    return C

### Define construction site

In [7]:
def create_construction_site(C, position = 'fixed'):
    
    if position == 'fixed':
        target_location = C.addFrame('construction_site', 'table')
        target_location.setShape(ry.ST.box, size = [.275, .275, .1])
        target_location.setRelativePosition([-.5, -.225, .0])
        target_location.setColor([1., 0.75, 0.])
        return target_location
    
    elif position == 'random':
        target_location = C.addFrame('construction_site', 'table')
        target_location.setShape(ry.ST.box, size = [.3, .3, .1])
        target_location.setRelativePosition([np.random.uniform(-0.5, 0.5), np.random.uniform(-0.5, 0.5), .0])
        target_location.setColor([1, 0.75, 0])
        return target_location
        

In [8]:
def create_pillar_target_postion(C, position = 'fixed', n_pillars = 2):
    
    if position == 'fixed':
        for i in range(0, n_pillars):
            target_pillar = C.addFrame('pillar_target_pos' + str(i+1), 'table')
            target_pillar.setShape(ry.ST.box, size=[.065, .065, .105])
            target_pillar.setRelativePosition([-.45 - (i / 10) , -.225, .0])
            target_pillar.setColor([0,1,0])
    
    elif position == 'random':
        for i in range(0, n_pillars):
            C = C.addFrame('pillar_target_pos' + str(i+1), 'table')
            target_pillar.setShape(ry.ST.box, size=[.065, .065, .105])
            target_pillar.setRelativePosition([np.random.uniform(-0.5, 0.5), np.random.uniform(-0.5, 0.5), .0])
            target_pillar.setColor([0,1,0])
        return C
        

### Define Komo problem

In [9]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))
C = create_n_boxes(C = C, N = 3, position = 'fixed')
goal = create_construction_site(C, position = 'fixed')
pillar_goal = create_pillar_target_postion(C, position = 'fixed')
C.view()

0

In [10]:
def gripper_to_block(C, bot, block = 'box1', prints = False):
    qHome = C.getJointState()

    komo = ry.KOMO(C, 1, 1, 0, False)

    komo.addObjective(
        times=[], 
        feature=ry.FS.jointState, 
        frames=[],
        type=ry.OT.sos, 
        scale=[1e-1], 
        target=qHome
    )
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], qHome)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)
    komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq)
    komo.addObjective([], ry.FS.positionDiff, ['l_gripper', block], ry.OT.eq, [1e1])
    komo.addObjective([], ry.FS.scalarProductXX, ['l_gripper', block], ry.OT.eq, [1e1], [0])
    komo.addObjective([], ry.FS.scalarProductXZ, ['l_gripper', block], ry.OT.eq, [1e1], [0])
    komo.addObjective([], ry.FS.distance, ['l_palm', block], ry.OT.ineq, [1e1])

    ret = ry.NLP_Solver(komo.nlp(), verbose=4) .solve()
    if prints == True:
        print(ret)
    
    q = komo.getPath()

    bot.moveTo(q[-1], 5)
    while bot.getTimeToEnd() > 0:
        bot.sync(C, .5)

In [11]:
def move_gripper_to(bot, add_frame_obj_str = 'target', offset = .075, prints=False):
    komo = ry.KOMO(C, 2, 1, 1, True)
    komo.addControlObjective([], 0, 1e-1)
    komo.addControlObjective([], 1, 1e0)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)
    #komo.addObjective([1], ry.FS.positionDiff, ['l_gripper', add_frame_obj_str], ry.OT.eq, [1e-1])
    #komo.addObjective([2], ry.FS.positionDiff, ['l_gripper', add_frame_obj_str], ry.OT.eq, [1e-1])
    komo.addObjective([1], ry.FS.positionDiff, ['l_gripper', add_frame_obj_str], ry.OT.eq, [1e-1])
    #komo.addObjective([1], ry.FS.positionDiff, ['l_gripper', add_frame_obj_str], ry.OT.eq, [1e-1], [0])
    
    #komo.addObjective([3], ry.FS.positionDiff, ['l_gripper', add_frame_obj_str], ry.OT.eq, [1e1])
    #komo.addObjective([], ry.FS.scalarProductXX, ['l_gripper', carry_item], ry.OT.eq, [1e1], [0])
    #komo.addObjective([], ry.FS.scalarProductXZ, ['l_gripper', carry_item], ry.OT.eq, [1e1], [0])
    #komo.addObjective([], ry.FS.distance, ['l_palm', carry_item], ry.OT.ineq, [1e1])
    komo.addObjective([2], ry.FS.positionRel, [add_frame_obj_str, 'cameraWrist'], ry.OT.eq, [1.], [.0, .0, offset])

    ret = ry.NLP_Solver(komo.nlp(), verbose=4) .solve()
    if prints == True:
        print(ret)
    
    q = komo.getPath()
    
    bot.moveTo(q[-1], 2)
    while bot.getTimeToEnd() > 0:
        bot.sync(C, .1)

In [12]:
def rotate_move_gripper_to(bot, add_frame_obj_str = 'target', offset = .075, prints=False):
    komo = ry.KOMO(C, 4, 1, 1, True)
    komo.addControlObjective([], 0, 1e-1)
    komo.addControlObjective([], 1, 1e0)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)
    komo.addObjective([1], ry.FS.positionDiff, ['l_gripper', add_frame_obj_str], ry.OT.eq, [1e-1])
    komo.addObjective([2], ry.FS.positionDiff, ['l_gripper', add_frame_obj_str], ry.OT.eq, [1e-1])
    komo.addObjective([2], ry.FS.positionDiff, ['l_gripper', add_frame_obj_str], ry.OT.eq, [1e-1], [0])
    komo.addObjective([2], ry.FS.positionDiff, ['l_gripper', add_frame_obj_str], ry.OT.eq, [1e-1], [0])
    
    komo.addObjective([3], ry.FS.positionDiff, ['l_gripper', add_frame_obj_str], ry.OT.eq, [1e1])
    #komo.addObjective([], ry.FS.scalarProductXX, ['l_gripper', carry_item], ry.OT.eq, [1e1], [0])
    #komo.addObjective([], ry.FS.scalarProductXZ, ['l_gripper', carry_item], ry.OT.eq, [1e1], [0])
    #komo.addObjective([], ry.FS.distance, ['l_palm', carry_item], ry.OT.ineq, [1e1])
    komo.addObjective([4], ry.FS.positionRel, [add_frame_obj_str, 'cameraWrist'], ry.OT.eq, [1.], [.0, .0, offset])

    ret = ry.NLP_Solver(komo.nlp(), verbose=4) .solve()
    if prints == True:
        print(ret)
    
    q = komo.getPath()
    
    bot.moveTo(q[-1])
    while bot.getTimeToEnd() > 0:
        bot.sync(C, .1)


In [13]:
def generate_waypoint_guides(C, start='', target='', offset=0, n_waypoints=5, interpolation='linear'):
    start_frame = C.getFrame(start)
    start_pos = np.array(start_frame.getPosition())
    target_frame = C.getFrame(target)
    target_pos = np.array(target_frame.getPosition())
    target_pos[2] += offset


    if interpolation == 'linear':
        t_vals = np.linspace(0, 1, n_waypoints)
        x_spline = Rbf([0, 1], [start_pos[0], target_pos[0]], function='linear')
        y_spline = Rbf([0, 1], [start_pos[1], target_pos[1]], function='linear')
        z_spline = Rbf([0, 1], [start_pos[2], target_pos[2]], function='linear')

        positions = np.column_stack([x_spline(t_vals), y_spline(t_vals), z_spline(t_vals)])
    
    elif interpolation == 'cubic':
        t_vals = np.linspace(0, 1, n_waypoints)
        x_spline = Rbf([0, 1], [start_pos[0], target_pos[0]], function='linear')
        y_spline = Rbf([0, 1], [start_pos[1], target_pos[1]], function='linear')
        z_spline = Rbf([0, 1], [start_pos[2], target_pos[2]], function='cubic')
        
        positions = np.column_stack([x_spline(t_vals), y_spline(t_vals), z_spline(t_vals)])


    for i, pos in enumerate(positions):
        way_name = 'way' + str(i+1)
        C.addFrame(way_name).setShape(ry.ST.marker, [.1, .1, .1]).setPosition(pos).setColor([1, .5, 1])

    return C


In [14]:
C = generate_waypoint_guides(C, start = 'box1', target = 'pillar_target_pos2', offset=0.120, n_waypoints=5, interpolation='linear')
C.view()

0

In [15]:
bot = ry.BotOp(C, useRealRobot = False)

## First block

In [29]:
gripper_open(C, bot)
gripper_to_block(C, bot, block='box1')
gripper_close_grasp(bot, target_obj = 'box1', force = 10.0,  widht = 0.045)
move_gripper_to(bot, add_frame_obj_str = 'way1')
move_gripper_to(bot, add_frame_obj_str = 'way2')
move_gripper_to(bot, add_frame_obj_str = 'way3')
move_gripper_to(bot, add_frame_obj_str = 'way4')
move_gripper_to(bot, add_frame_obj_str = 'way5')
gripper_open(C, bot)
bot.home(C)

====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1
----newton---- initial point f(x):0.164169 alpha:1 beta:1
--newton-- it:   1  |Delta|:  0.0170007  alpha:          1  evals:   2  f(y): 0.00246106  ACCEPT
--newton-- stopping: 'absMax(Delta)<options.stopTolerance'
==nlp== it:   0  evals:   2  A(x): 0.00246106  f:2.50895e-06  g:  0.0350064  h:  0.0403855  |x-x'|:  0.0170007 	stop:DeltaConverge
==nlp== it:   1  evals:   2  A(x):  0.0172123  mu:5
--newton-- it:   2  |Delta|: 0.00109232  alpha:          1  evals:   3  f(y):  0.0171644  ACCEPT
--newton-- stopping: 'absMax(Delta)<options.stopTolerance'
==nlp== it:   1  evals:   3  A(x):  0.0171644  f:  2.486e-06  g:  0.0350292  h:  0.0365511  |x-x'|: 0.00109232 	stop:DeltaConverge
==nlp== StoppingCriterion Delta<0.01
----newton---- final f(x):0.0171644
====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1
----newton---- initial point f(x):0.0377403 alpha:1 beta:1


### Second Block

In [30]:
C = generate_waypoint_guides(C, start = 'box2', target = 'pillar_target_pos1', offset=0.120, n_waypoints=5, interpolation='linear')
bot = ry.BotOp(C, useRealRobot = False)
C.view()

0

-- WARNING:kin.cpp:addFrame:192(-1) frame already exists! returning existing without modifications!
-- WARNING:kin.cpp:addFrame:192(-1) frame already exists! returning existing without modifications!
-- WARNING:kin.cpp:addFrame:192(-1) frame already exists! returning existing without modifications!
-- WARNING:kin.cpp:addFrame:192(-1) frame already exists! returning existing without modifications!
-- WARNING:kin.cpp:addFrame:192(-1) frame already exists! returning existing without modifications!
-- bot.cpp:~BotOp:118(0) shutting down BotOp...
-- simulation.cpp:~BotThreadedSim:56(0) shutting down SimThread
-- simulation.cpp:~Simulation:148(0) shutting down Simulation


In [31]:
gripper_to_block(C, bot, block='box2')
gripper_close_grasp(bot, target_obj = 'box1', force = 10.0,  widht = 0.045)
move_gripper_to(bot, add_frame_obj_str = 'way1')
move_gripper_to(bot, add_frame_obj_str = 'way2')
move_gripper_to(bot, add_frame_obj_str = 'way3')
move_gripper_to(bot, add_frame_obj_str = 'way4')
move_gripper_to(bot, add_frame_obj_str = 'way5')
gripper_open(C, bot)
bot.home(C)

====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1
----newton---- initial point f(x):57.6184 alpha:1 beta:1
--newton-- it:   1  |Delta|:        0.2  alpha:          1  evals:   2  f(y):    34.8183  ACCEPT
--newton-- it:   2  |Delta|:        0.2  alpha:          1  evals:   3  f(y):     14.268  ACCEPT
--newton-- it:   3  |Delta|:        0.2  alpha:          1  evals:   4  f(y):    6.18065  ACCEPT
--newton-- it:   4  |Delta|:        0.2  alpha:          1  evals:   5  f(y):     2.3709  ACCEPT
--newton-- it:   5  |Delta|:        0.2  alpha:          1  evals:   6  f(y):    0.51241  ACCEPT
--newton-- it:   6  |Delta|:   0.171129  alpha:          1  evals:   7  f(y):  0.0402382  ACCEPT
--newton-- it:   7  |Delta|:  0.0205223  alpha:          1  evals:   8  f(y):  0.0322902  ACCEPT
--newton-- stopping: 'absMax(Delta)<options.stopTolerance'
==nlp== it:   0  evals:   8  A(x):  0.0322902  f:  0.0295734  g:  0.0328782  h:  0.0479485  |x-x'|:   0.963067 	s

### 3rd Block

In [32]:
box_1 = C.getFrame('box1')
box_2 = C.getFrame('box2')

box_pos1 = box_1.getPosition()
box_pos2 = box_2.getPosition()
offset = 0.075
middle_between_towers = np.mean([box_pos1, box_pos2], 0) + [0, 0, offset]
C.addFrame('block3_target').setShape(ry.ST.marker, [.1, .1, .1]).setPosition(middle_between_towers).setColor([1, .5, 1])
C.view()

0

In [33]:
C = generate_waypoint_guides(C, start = 'box3', target = 'block3_target', offset=0.0125, n_waypoints=5, interpolation='linear')
bot = ry.BotOp(C, useRealRobot = False)
C.view()

-- WARNING:kin.cpp:addFrame:192(-1) frame already exists! returning existing without modifications!
-- WARNING:kin.cpp:addFrame:192(-1) frame already exists! returning existing without modifications!
-- WARNING:kin.cpp:addFrame:192(-1) frame already exists! returning existing without modifications!
-- WARNING:kin.cpp:addFrame:192(-1) frame already exists! returning existing without modifications!
-- WARNING:kin.cpp:addFrame:192(-1) frame already exists! returning existing without modifications!
-- bot.cpp:~BotOp:118(0) shutting down BotOp...
-- simulation.cpp:~BotThreadedSim:56(0) shutting down SimThread
-- simulation.cpp:~Simulation:148(0) shutting down Simulation


0

In [92]:
def rotate_move_gripper_to(bot, add_frame_obj_str = 'target', offset = .075, prints=False):
    komo = ry.KOMO(C, 4, 1, 1, True)
    komo.addControlObjective([], 0, 1e-1)
    komo.addControlObjective([], 1, 1e0)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)
    komo.addObjective([], ry.FS.positionDiff, ['l_gripper', add_frame_obj_str], ry.OT.eq, [1e-1])
    komo.addObjective([], ry.FS.scalarProductXX, ['l_gripper', add_frame_obj_str], ry.OT.eq, [1e1], [-1])
    komo.addObjective([], ry.FS.scalarProductYY, ['l_gripper', add_frame_obj_str], ry.OT.eq, [1e1], [0])

    #komo.addObjective([], ry.FS.scalarProductXZ, ['l_gripper', add_frame_obj_str], ry.OT.eq, [1e1], [1])

    ret = ry.NLP_Solver(komo.nlp(), verbose=4) .solve()
    if prints == True:
        print(ret)
    
    q = komo.getPath()
    
    bot.moveTo(q[-1])
    while bot.getTimeToEnd() > 0:
        bot.sync(C, .1)

rotate_move_gripper_to(bot, add_frame_obj_str = 'block3_target')

In [65]:
rotate_move_gripper_to(bot, add_frame_obj_str = 'block3_target')

In [34]:
gripper_to_block(C, bot, block='box3')
gripper_close_grasp(bot, target_obj = 'box3', force = 10.0,  widht = 0.045)
move_gripper_to(bot, add_frame_obj_str = 'way1')
move_gripper_to(bot, add_frame_obj_str = 'way2')
move_gripper_to(bot, add_frame_obj_str = 'way3')

====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1
----newton---- initial point f(x):56.7259 alpha:1 beta:1
--newton-- it:   1  |Delta|:        0.2  alpha:          1  evals:   2  f(y):    34.3993  ACCEPT
--newton-- it:   2  |Delta|:        0.2  alpha:          1  evals:   3  f(y):    14.6859  ACCEPT
--newton-- it:   3  |Delta|:        0.2  alpha:          1  evals:   4  f(y):    6.26611  ACCEPT
--newton-- it:   4  |Delta|:        0.2  alpha:          1  evals:   5  f(y):    2.43945  ACCEPT
--newton-- it:   5  |Delta|:        0.2  alpha:          1  evals:   6  f(y):   0.556909  ACCEPT
--newton-- it:   6  |Delta|:   0.182435  alpha:          1  evals:   7  f(y):  0.0450438  ACCEPT
--newton-- it:   7  |Delta|:  0.0218208  alpha:          1  evals:   8  f(y):  0.0348774  ACCEPT
--newton-- stopping: 'absMax(Delta)<options.stopTolerance'
==nlp== it:   0  evals:   8  A(x):  0.0348774  f:  0.0314301  g:  0.0375545  h:  0.0564326  |x-x'|:   0.958065 	s

In [41]:
move_gripper_to(bot, add_frame_obj_str = 'way4')
move_gripper_to(bot, add_frame_obj_str = 'way5')
#move_gripper_to(bot, add_frame_obj_str = 'way3')

====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1
----newton---- initial point f(x):0.220562 alpha:1 beta:1
--newton-- it:   1  |Delta|:  0.0772499  alpha:          1  evals:   2  f(y):    0.19641  ACCEPT
--newton-- it:   2  |Delta|:  0.0355083  alpha:          1  evals:   3  f(y):   0.189713  ACCEPT
--newton-- it:   3  |Delta|:  0.0258922  alpha:          1  evals:   4  f(y):   0.185966  ACCEPT
--newton-- stopping: 'absMax(Delta)<options.stopTolerance'
==nlp== it:   0  evals:   4  A(x):   0.185966  f:  0.0750391  g:          0  h:   0.510923  |x-x'|:    0.13015 	stop:DeltaConverge
==nlp== it:   1  evals:   4  A(x):   0.851529  mu:5
--newton-- it:   4  |Delta|:        0.2  alpha:          1  evals:   5  f(y):   0.562243  ACCEPT
--newton-- it:   5  |Delta|:  0.0938113  alpha:          1  evals:   6  f(y):   0.460478  ACCEPT
--newton-- it:   6  |Delta|:  0.0416393  alpha:          1  evals:   7  f(y):   0.442206  ACCEPT
--newton-- it:   7  |Delta

### Clear config

In [87]:
C.clear()